In [1]:
import torch
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [2]:
import torchvision.transforms as T

T_norm = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

T_tensor = T.Compose([
    T.ToTensor()
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def getPredict(a):
    pred =a['out'].data.cpu().numpy()
    return np.argmax(pred, axis=1)[0]

def get_segment_labels(image, model, device, transform):
    # transform the image to tensor and load into computation device
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)
    return outputs

def getPourcentage(a):
    return np.sum(a)/(a.shape[0]*a.shape[1])

In [3]:
# Load the trained model 
model = torch.load(r'C:\Users\A60026184\Desktop\ModelA\Results\weights_Aaug5_v1.pt')
model.eval()

print('DONE Loading models...')

DONE Loading models...


In [8]:
img_files = glob(r'C:\Users\A60026184\Desktop\ModelA\Todo\*.*')
outpath = r'E:\Aurélie'

In [6]:
p_pred = []
name = []

for f in img_files:
    filename = f.split('\\')[-1]
    fichier = filename.split('.')[0][3:]
    
    name.append(fichier)
    
    # read the image
    image = Image.open(f)
    result = get_segment_labels(image, model, device, T_norm)
    pred = getPredict(result)
    
    p_pred.append(getPourcentage(pred))

In [9]:
import pandas as pd
df_pred = pd.DataFrame(p_pred, columns= ['Predictions'])
df_name = pd.DataFrame(name, columns=['filename'])

df = df_name.join(df_pred)
df = df.groupby(['filename']).mean().reset_index()
df.to_csv(outpath+'\\Todo_predA.csv')